## Reason for the missing alpha index: 
- **SMA**: 9, 23, 24, 28, 47, 57, 63, 67, 68, 72, 79, 81, 82, 96, 102, 109, 111, 122, 135, 151, 152, 155, 158, 160, 162, 164, 169, 173, 174, 188
- **Too Complicated ': ?'**: 4, 49, 50, 51, 55, 86, 94, 98, 109, 112, 128, 129, 137, 143, 146, 167, 172, 180, 186, 187
- **Error from the report**:10, 127, 144, 165, 166, 183, 190
- **REGBETA() Definition**: 21, 116, 147, 149
- **SMEAN() Definition**:22
- **MKY SMB HML**: 30
- **BANCKMARKINDEXCLOSE**: 75, 181, 182, 
- **RANK 函数写得有问题**：83, 99

In [34]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
%matplotlib inline

a = 5
b = 0 if a<0 else 1 if a<1 else 5
b

5

In [2]:
BTC_data = pd.read_pickle('BTC_data.pickle')

In [3]:
BTC_data.columns

Index(['price_close', 'price_high', 'price_low', 'price_open',
       'volume_traded'],
      dtype='object')

In [4]:
from BTC_Alpha_func import *

### **先罗列具体alpha因子的计算之前，先按照国泰研报中的表15的顺序写好常用的函数**

In [5]:
OPEN = BTC_data['price_open']
HIGH = BTC_data['price_high']
LOW = BTC_data['price_low']
CLOSE = BTC_data['price_close']
VOLUME = BTC_data['volume_traded']
VWAP = CLOSE
RET = RET(CLOSE)
AMOUNT = VOLUME * CLOSE
DTM = (OPEN.values <= DELAY(OPEN, 1).values) \
            * MAX(HIGH - OPEN, OPEN - DELAY(OPEN, 1))
DBM = (OPEN >= DELAY(OPEN, 1).values) \
            * MAX(OPEN - LOW, OPEN - DELAY(OPEN, 1))


In [6]:
Alpha1 = -1 * CORR(RANK(DELTA(LOG(VOLUME),1)), RANK((CLOSE-OPEN)/OPEN), 6)
Alpha1.head()

2018-02-01 00:00:10    1.000000
2018-02-01 00:00:20    1.000000
2018-02-01 00:00:30    0.986811
2018-02-01 00:00:40    0.990591
2018-02-01 00:02:40   -0.074268
dtype: float64

In [7]:
##会出现NAN，因为HIGH = LOW
Alpha2 = (-1 * DELTA((((CLOSE - LOW) - (HIGH - CLOSE)) / (HIGH - LOW)), 1)).fillna(0)
Alpha2.tail()

2018-04-14 02:03:20    0.000000
2018-04-14 02:03:30    1.872679
2018-04-14 02:03:40   -1.874976
2018-04-14 02:04:00    0.001596
2018-04-14 02:04:10    0.000000
dtype: float64

In [8]:
cache = CLOSE - ((~(CLOSE>DELAY(CLOSE,1)))*MAX(HIGH,DELAY(CLOSE,1)) + (~(CLOSE>DELAY(CLOSE,1)))*MAX(HIGH,DELAY(CLOSE,1)))
#((CLOSE>DELAY(CLOSE,1))*MIN(LOW,DELAY(CLOSE,1)) + (~(CLOSE>DELAY(CLOSE,1)))*MAX(HIGH,DELAY(CLOSE,1)))
Alpha3 = SUM((~(CLOSE==DELAY(CLOSE,1)) * cache), 6)
Alpha3.tail()

2018-04-14 02:03:20   -16133.8756
2018-04-14 02:03:30   -32244.8856
2018-04-14 02:03:40   -32230.9293
2018-04-14 02:04:00   -32235.9894
2018-04-14 02:04:10   -32230.9393
dtype: float64

In [9]:
#Alpha4 = ((((SUM(CLOSE, 8) / 8) + STD(CLOSE, 8)) < (SUM(CLOSE, 2) / 2)) ? (-1 * 1) : (((SUM(CLOSE, 2) / 2) < ((SUM(CLOSE, 8) / 8) - STD(CLOSE, 8))) ? 1 : (((1 < (VOLUME / MEAN(VOLUME,20))) || ((VOLUME / MEAN(VOLUME,20)) == 1)) ? 1 : (-1 * 1))))

In [10]:
Alpha5 = (-1 * TSMAX(CORR(TSRANK(VOLUME, 5), TSRANK(HIGH, 5), 5), 3))
Alpha5.tail()

2018-04-14 02:03:20   -0.879049
2018-04-14 02:03:30   -0.469018
2018-04-14 02:03:40   -0.469018
2018-04-14 02:04:00   -0.469018
2018-04-14 02:04:10    0.201008
dtype: float64

In [11]:
Alpha6 = (RANK(SIGN(DELTA((((OPEN * 0.85) + (HIGH * 0.15))), 4)))* (-1))
Alpha6.tail()

2018-04-14 02:03:20    -48576.5
2018-04-14 02:03:30    -48576.5
2018-04-14 02:03:40    -48576.5
2018-04-14 02:04:00    -48576.5
2018-04-14 02:04:10   -147500.5
dtype: float64

In [12]:
'''if VWAP is defined as CLOSE, then MAX() have no funtion but to reproduce 3, so as for the rank'''
Alpha7 = ((RANK(MAX((VWAP - CLOSE), 3)) + RANK(MIN((VWAP - CLOSE), 3))) * RANK(DELTA(VOLUME, 3)))
#Alpha7.head()

#Alpha7.tail()
Alpha8 = RANK(DELTA(((((HIGH + LOW) / 2) * 0.2) + (VWAP * 0.8)), 4) * -1)
Alpha8.head()

2018-02-01 00:00:10     97447.5
2018-02-01 00:00:20    104534.5
2018-02-01 00:00:30    104534.5
2018-02-01 00:00:40    104536.0
2018-02-01 00:02:40    180411.0
dtype: float64

In [13]:
Alpha8 = RANK(DELTA(((((HIGH + LOW) / 2) * 0.2) + (VWAP * 0.8)), 4) * -1)

In [14]:
'''
SMA is not defined yet, pass
Alpha9 = SMA(((HIGH+LOW)/2-(DELAY(HIGH,1)+DELAY(LOW,1))/2)*(HIGH-LOW)/VOLUME,7,2)
'''

'\nSMA is not defined yet, pass\nAlpha9 = SMA(((HIGH+LOW)/2-(DELAY(HIGH,1)+DELAY(LOW,1))/2)*(HIGH-LOW)/VOLUME,7,2)\n'

In [31]:
'''
原文是“(RANK(MAX(((RET < 0) ? STD(RET, 20) : CLOSE)^2),5))”,
这么写有两个问题，一个是RANK传进去了两个变量MAX() & 5, 而实际上该函数只是负责将DataFrame进行排序，TSRANK()才会涉及到窗口的设置；
第二个问题是MAX()只传入了一个参数，而该函数实现的功能是比较两个DataFrame同一位置上的两个函数，因此这里我先将5从RANK()中放到MAX()中去

'''

Alpha10 = RANK(MAX((STD(RET, 20) * (RET < 0) + CLOSE**2 * ~(RET < 0) * CLOSE)**2, 5))
#Alpha10.tail()


In [16]:
Alpha11 = SUM(((CLOSE-LOW)-(HIGH-CLOSE))/(HIGH-LOW)*VOLUME,6)
Alpha11.tail()

2018-04-14 02:03:20     5.948268
2018-04-14 02:03:30     0.684689
2018-04-14 02:03:40    11.862410
2018-04-14 02:04:00     8.774614
2018-04-14 02:04:10     8.774614
dtype: float64

In [17]:
Alpha12 = (RANK((OPEN - (SUM(VWAP, 10) / 10)))) * (-1 * (RANK(ABS((CLOSE - VWAP)))))
Alpha12.tail()

2018-04-14 02:03:20   -4.889262e+09
2018-04-14 02:03:30   -4.606110e+09
2018-04-14 02:03:40   -2.113081e+09
2018-04-14 02:04:00   -8.422405e+09
2018-04-14 02:04:10   -8.838822e+09
dtype: float64

In [18]:
Alpha13 = (((HIGH * LOW)**0.5) - VWAP)
Alpha13.tail()

2018-04-14 02:03:20    0.000000
2018-04-14 02:03:30    0.070000
2018-04-14 02:03:40   -4.131510
2018-04-14 02:04:00   -6.227315
2018-04-14 02:04:10    0.000000
dtype: float64

In [19]:
Alpha14 = CLOSE-DELAY(CLOSE,5)
Alpha14.tail()

2018-04-14 02:03:20   -9.2400
2018-04-14 02:03:30   -4.7533
2018-04-14 02:03:40   -9.4004
2018-04-14 02:04:00   -5.0501
2018-04-14 02:04:10   -5.0290
Name: price_close, dtype: float64

In [20]:
Alpha15 = OPEN/DELAY(CLOSE,1)-1

In [21]:
Alpha16 = (-1 * TSMAX(RANK(CORR(RANK(VOLUME), RANK(VWAP), 5)), 5))
Alpha16.tail()

2018-04-14 02:03:20   -141858.0
2018-04-14 02:03:30   -141858.0
2018-04-14 02:03:40   -115944.0
2018-04-14 02:04:00   -115944.0
2018-04-14 02:04:10   -115944.0
dtype: float64

In [22]:
Alpha17 = RANK((VWAP - MAX(VWAP, 15)))**DELTA(CLOSE, 5)

In [23]:
Alpha18 = CLOSE/DELAY(CLOSE,5)


Alpha19 = (CLOSE-DELAY(CLOSE,5))/DELAY(CLOSE,5) * (CLOSE<DELAY(CLOSE,5)) + \
        ~(CLOSE<DELAY(CLOSE,5)) * (0*(CLOSE==DELAY(CLOSE,5))+ ~(CLOSE==DELAY(CLOSE,5))* (CLOSE-DELAY(CLOSE,5))/CLOSE)
Alpha19.tail()

2018-04-14 02:03:20   -0.001146
2018-04-14 02:03:30   -0.000590
2018-04-14 02:03:40   -0.001166
2018-04-14 02:04:00   -0.000627
2018-04-14 02:04:10   -0.000624
Name: price_close, dtype: float64

In [24]:
Alpha20 = (CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)*100

In [25]:
#a = pd.Series(np.linspace(1,6,6))
'''原文中没有设置拟合的窗口，我根据含义设置为’6‘，我发现所有的REGBETA()都是计算的一个CLOSE/OPEN与一个sequence的拟合与方差，即要求一个rolling window和一个fixed window去拟合，暂时没有想到比较好的解决方法，暂时搁置'''
#Alpha21 = REGBETA(MEAN(CLOSE,6),a,6)

corrvariance function (调用numpy)

SyntaxError: invalid syntax (<ipython-input-25-b1eb0cadd8d1>, line 5)

In [26]:
'''SMEAN(**arg, arg1, arg2) is not defined, we must figure out its definition first'''
#Alpha22 = SMEAN(((CLOSE-MEAN(CLOSE,6))/MEAN(CLOSE,6)-DELAY((CLOSE-MEAN(CLOSE,6))/MEAN(CLOSE,6),3)),12,1)

'SMEAN(**arg, arg1, arg2) is not defined, we must figure out its definition first'

In [27]:
Alpha25 = ((-1 * RANK((DELTA(CLOSE, 7) * (1 - RANK(DECAYLINEAR((VOLUME / MEAN(VOLUME,20)), 9)))))) * (1 + RANK(SUM(RET, 250))))

Alpha25.tail()

2018-04-14 02:03:20   -2.393315e+10
2018-04-14 02:03:30   -2.389261e+10
2018-04-14 02:03:40   -2.592001e+10
2018-04-14 02:04:00   -1.808293e+10
2018-04-14 02:04:10   -2.473331e+10
dtype: float64

In [28]:
Alpha26 = ((((SUM(CLOSE, 7) / 7) - CLOSE)) + ((CORR(VWAP, DELAY(CLOSE, 5), 230))))
Alpha26.tail()

2018-04-14 02:03:20    8.224546
2018-04-14 02:03:30    7.019255
2018-04-14 02:03:40    5.708110
2018-04-14 02:04:00    1.305041
2018-04-14 02:04:10    0.599883
Name: price_close, dtype: float64

In [29]:
Alpha27 = WMA((CLOSE-DELAY(CLOSE,3))/DELAY(CLOSE,3)*100+(CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)*100,12)
Alpha27.tail()

2018-04-14 02:03:20    0.050277
2018-04-14 02:03:30    0.006907
2018-04-14 02:03:40   -0.024901
2018-04-14 02:04:00   -0.030733
2018-04-14 02:04:10   -0.039074
Name: price_close, dtype: float64

In [ ]:
Alpha29 = (CLOSE-DELAY(CLOSE,6))/DELAY(CLOSE,6)*VOLUME
Alpha29.tail()

In [ ]:
Alpha31 = (CLOSE-MEAN(CLOSE,12))/MEAN(CLOSE,12)*100

In [ ]:
Alpha32 = (-1 * SUM(RANK(CORR(RANK(HIGH), RANK(VOLUME), 3)), 3))

In [ ]:
Alpha33 = ((((-1 * TSMIN(LOW, 5)) + DELAY(TSMIN(LOW, 5), 5)) * RANK(((SUM(RET, 240) - SUM(RET, 20)) / 220))) * TSRANK(VOLUME, 5))

In [ ]:
Alpha34 = MEAN(CLOSE,12)/CLOSE

In [ ]:
Alpha35 = (MIN(RANK(DECAYLINEAR(DELTA(OPEN, 1), 15)), RANK(DECAYLINEAR(CORR((VOLUME), ((OPEN * 0.65) + (OPEN *0.35)), 17),7))) * -1)

In [ ]:
Alpha36 = RANK(SUM(CORR(RANK(VOLUME), RANK(VWAP), 6), 2))

In [ ]:
Alpha37 = (-1 * RANK(((SUM(OPEN, 5) * SUM(RET, 5)) - DELAY((SUM(OPEN, 5) * SUM(RET, 5)), 10))))

In [ ]:
Alpha38 = ((SUM(HIGH, 20) / 20) < HIGH) * (-1 * DELTA(HIGH, 2)) + 0 * ~(SUM(HIGH, 20) / 20 < HIGH)

In [ ]:
Alpha39 = ((RANK(DECAYLINEAR(DELTA((CLOSE), 2),8)) - RANK(DECAYLINEAR(CORR(((VWAP * 0.3) + (OPEN * 0.7)), SUM(MEAN(VOLUME,180), 37), 14), 12))) * -1)

In [1]:
Alpha40 = SUM(CLOSE>DELAY(CLOSE,1)*VOLUME+ 0*~(CLOSE>DELAY(CLOSE,1)),26)\
               /SUM(CLOSE<=DELAY(CLOSE,1)*VOLUME+0*~(CLOSE<=DELAY(CLOSE,1)), 26)*100

NameError: name 'SUM' is not defined

In [ ]:
Alpha41 = (RANK(MAX(DELTA((VWAP), 3), 5))* -1)

In [ ]:
#HIGH
Alpha42 = ((-1 * RANK(STD(HIGH, 10))) * CORR(HIGH, VOLUME, 10))

In [ ]:
Alpha43 = SUM(VOLUME * (CLOSE>DELAY(CLOSE,1)) + ~(CLOSE>DELAY(CLOSE,1)) * (CLOSE<DELAY(CLOSE,1)*(-VOLUME) + 0 * ~(CLOSE<DELAY(CLOSE,1))),6)

Alpha43.tail()

In [ ]:
Alpha44 = (TSRANK(DECAYLINEAR(CORR(((LOW )), MEAN(VOLUME,10), 7), 6),4) + TSRANK(DECAYLINEAR(DELTA((VWAP), 3), 10), 15))

In [ ]:
Alpha45 = (RANK(DELTA((((CLOSE * 0.6) + (OPEN *0.4))), 1)) * RANK(CORR(VWAP, MEAN(VOLUME,150), 15)))

In [ ]:
Alpha46 = (MEAN(CLOSE,3)+MEAN(CLOSE,6)+MEAN(CLOSE,12)+MEAN(CLOSE,24))/(4*CLOSE)

In [ ]:
Alpha48 = (-1*((RANK(((SIGN((CLOSE - DELAY(CLOSE, 1))) + SIGN((DELAY(CLOSE, 1) - DELAY(CLOSE, 2)))) + SIGN((DELAY(CLOSE, 2) - DELAY(CLOSE, 3)))))) * SUM(VOLUME, 5)) / SUM(VOLUME, 20))

In [ ]:
##Alpha49-51式子过于冗长 暂时搁置。。。

In [ ]:
##把字节中的L替换为LOW
Alpha52 = SUM(MAX(0,HIGH-DELAY((HIGH+LOW+CLOSE)/3,1)),26)/SUM(MAX(0,DELAY((HIGH+LOW+CLOSE)/3,1) - LOW),26)* 100

In [ ]:
Alpha53 = COUNT(CLOSE, CLOSE>DELAY(CLOSE,1),12)/12*100

In [ ]:
#STD() missing 1 required positional argument: 'n'，姑且设置为’5‘

Alpha54 = (-1 * RANK((STD(ABS(CLOSE - OPEN), 5) + (CLOSE - OPEN)) + CORR(CLOSE, OPEN, 10)))

In [ ]:
##Alpha55太美好了 = =

In [ ]:
Alpha56 = (RANK((OPEN - TSMIN(OPEN, 12))) < RANK((RANK(CORR(SUM(((HIGH + LOW)/2), 19), SUM(MEAN(VOLUME,40), 19), 13))**5)))

In [ ]:
Alpha58 = COUNT(CLOSE, CLOSE>DELAY(CLOSE,1),20)/20*100

In [ ]:
Alpha59 = SUM((CLOSE==DELAY(CLOSE,1))*0+(CLOSE!=DELAY(CLOSE,1))*CLOSE \
        - ((CLOSE>DELAY(CLOSE,1))*MIN(LOW,DELAY(CLOSE,1)) + ~(CLOSE>DELAY(CLOSE,1)*MAX(HIGH,DELAY(CLOSE,1)))), 20)
     

In [ ]:
Alpha60 = SUM(((CLOSE-LOW)-(HIGH-CLOSE))/(HIGH-LOW)*VOLUME,20)

In [ ]:
Alpha61 = (MAX(RANK(DECAYLINEAR(DELTA(VWAP, 1),12)),RANK(DECAYLINEAR(RANK(CORR((LOW),MEAN(VOLUME,80), 8)), 17))) * -1)

In [ ]:
Alpha62 = (-1 * CORR(HIGH, RANK(VOLUME), 5))

In [ ]:
Alpha64 = (MAX(RANK(DECAYLINEAR(CORR(RANK(VWAP), RANK(VOLUME), 4),4)),RANK(DECAYLINEAR(MAX(CORR(RANK(CLOSE), RANK(MEAN(VOLUME,60)), 4), 13), 14))) * -1)

In [ ]:
Alpha65 = MEAN(CLOSE,6)/CLOSE

In [ ]:
Alpha67 = (CLOSE-MEAN(CLOSE,6))/MEAN(CLOSE,6)*100

In [ ]:
Alpha69 = (SUM(DTM,20)>SUM(DBM,20)) * (SUM(DTM,20)-SUM(DBM,20))/SUM(DTM,20) + \
        ~(SUM(DTM,20)>SUM(DBM,20))*((SUM(DTM,20)==SUM(DBM,20))*0 + (SUM(DTM,20)!=SUM(DBM,20))*(SUM(DTM,20)-SUM(DBM,20))/SUM(DBM,20))
Alpha69.tail()

In [ ]:

Alpha70 = STD(AMOUNT,6)

In [ ]:
Alpha71 = (CLOSE-MEAN(CLOSE,24))/MEAN(CLOSE,24)*100

In [ ]:
Alpha73 = ((TSRANK(DECAYLINEAR(DECAYLINEAR(CORR((CLOSE), VOLUME,10),16),4),5) -RANK(DECAYLINEAR(CORR(VWAP, MEAN(VOLUME,30), 4),3))) * -1)

In [ ]:
Alpha74 = (RANK(CORR(SUM(((LOW * 0.35) + (VWAP*0.65)), 20), SUM(MEAN(VOLUME,40),20), 7)) +RANK(CORR(RANK(VWAP), RANK(VOLUME), 6))) 

In [ ]:
Alpha76 = STD(ABS((CLOSE/DELAY(CLOSE,1)-1))/VOLUME,20)/MEAN(ABS((CLOSE/DELAY(CLOSE,1)-1))/VOLUME,20)

In [ ]:
Alpha77 = MIN(RANK(DECAYLINEAR(((((HIGH + LOW) / 2) + HIGH) - (VWAP+HIGH)),20)),RANK(DECAYLINEAR(CORR(((HIGH + LOW) / 2), MEAN(VOLUME,40), 3), 6)))

In [ ]:
"这里将MA()替换为MEAN()"
Alpha78 = ((HIGH+LOW+CLOSE)/3-MEAN((HIGH+LOW+CLOSE)/3,12))/(0.015*MEAN(ABS(CLOSE-MEAN((HIGH+LOW+CLOSE)/3,12)),12))

In [ ]:
Alpha80 = (VOLUME-DELAY(VOLUME,5))/DELAY(VOLUME,5)*100

In [30]:
Alpha83 = (-1 * RANK(COVIANCE(RANK(HIGH), RANK(VOLUME), 5)))

TypeError: count() got an unexpected keyword argument 'others'

In [ ]:
Alpha84 = SUM((CLOSE>DELAY(CLOSE,1)*VOLUME+ \
               ~(CLOSE>DELAY(CLOSE,1))*(CLOSE<DELAY(CLOSE,1)*(-VOLUME)+ 0* ~(CLOSE<DELAY(CLOSE,1)))),20)

In [ ]:
Alpha85 = (TSRANK((VOLUME / MEAN(VOLUME,20)), 20) * TSRANK((-1 * DELTA(CLOSE, 7)), 8))

In [ ]:
#Alpha86我选择狗带

In [ ]:
Alpha87 = ((RANK(DECAYLINEAR(DELTA(VWAP, 4), 7)) + TSRANK(DECAYLINEAR(((((LOW * 0.9) + (LOW * 0.1)) - VWAP) / (OPEN - ((HIGH + LOW) / 2))), 11), 7)) * -1)

In [ ]:
Alpha88 = (CLOSE-DELAY(CLOSE,20))/DELAY(CLOSE,20)*100

In [ ]:
Alpha90 = ( RANK(CORR(RANK(VWAP), RANK(VOLUME), 5)) * -1)

In [ ]:
Alpha91 = ((RANK((CLOSE - MAX(CLOSE, 5)))*RANK(CORR((MEAN(VOLUME,40)), LOW, 5))) * -1)

In [ ]:
Alpha92 = MAX(RANK(DECAYLINEAR(DELTA(((CLOSE* 0.35)+ (VWAP*0.65)),2),3)),\
              TSRANK(DECAYLINEAR(ABS(CORR((MEAN(VOLUME,180)), CLOSE, 13)), 5), 15)) * -1

In [ ]:
Alpha93 = SUM((OPEN>=DELAY(OPEN,1)*0+ ~(OPEN>=DELAY(OPEN,1))*MAX((OPEN-LOW),(OPEN-DELAY(OPEN,1)))),20)

In [ ]:
#Alpha94,98 : ?

In [ ]:
Alpha95 = STD(AMOUNT,20)

In [ ]:
Alpha97 = STD(VOLUME,10)

In [ ]:
'''similiar as Alpha83, rank函数有些问题'''
#Alpha99 = (-1 * RANK(COVIANCE(RANK(CLOSE), RANK(VOLUME), 5)))

In [ ]:
Alpha100 = STD(VOLUME,20)

In [ ]:
Alpha101 = ((RANK(CORR(CLOSE, SUM(MEAN(VOLUME,30), 37), 15)) < RANK(CORR(RANK(((HIGH * 0.1) + (VWAP * 0.9))), RANK(VOLUME), 11))) * -1)

In [ ]:
Alpha103 = ((20-LOWDAY(LOW,20))/20)*100

In [ ]:
Alpha104 = (-1 * (DELTA(CORR(HIGH, VOLUME, 5), 5) * RANK(STD(CLOSE, 20))))

In [ ]:
Alpha105 = (-1 * CORR(RANK(OPEN), RANK(VOLUME), 10))

In [ ]:
Alpha106 = CLOSE-DELAY(CLOSE,20)

In [ ]:
Alpha107 = (((-1 * RANK((OPEN - DELAY(HIGH, 1)))) * RANK((OPEN - DELAY(CLOSE, 1)))) * RANK((OPEN - DELAY(LOW, 1))))

In [ ]:
Alpha108 = ((RANK((HIGH - MIN(HIGH, 2)))**RANK(CORR((VWAP), (MEAN(VOLUME,120)), 6))) * -1)

In [ ]:
Alpha110 = SUM(MAX(0,HIGH-DELAY(CLOSE,1)),20)/SUM(MAX(0,DELAY(CLOSE,1)-LOW),20)*100

In [ ]:
#Alpha109,112 : ?

In [ ]:
Alpha113 = (-1 * ((RANK((SUM(DELAY(CLOSE, 5), 20) / 20)) * CORR(CLOSE, VOLUME, 2)) * RANK(CORR(SUM(CLOSE, 5), SUM(CLOSE, 20), 2))))

In [ ]:
Alpha114 = ((RANK(DELAY(((HIGH - LOW) / (SUM(CLOSE, 5) / 5)), 2)) * RANK(RANK(VOLUME))) / (((HIGH - LOW) / (SUM(CLOSE, 5) / 5)) / (VWAP - CLOSE)))

In [ ]:
Alpha115 = RANK(CORR(((HIGH * 0.9) + (CLOSE * 0.1)), MEAN(VOLUME,30), 10))**RANK(CORR(TSRANK(((HIGH + LOW) / 2), 4), TSRANK(VOLUME, 10), 7))

In [ ]:
Alpha117 = ((TSRANK(VOLUME, 32) * (1 - TSRANK(((CLOSE + HIGH) - LOW), 16))) * (1 - TSRANK(RET, 32)))

In [ ]:
Alpha118 = SUM(HIGH-OPEN,20)/SUM(OPEN-LOW,20)*100

In [ ]:
Alpha119 = (RANK(DECAYLINEAR(CORR(VWAP, SUM(MEAN(VOLUME,5), 26), 5),7))-RANK(DECAYLINEAR(TSRANK(MIN(CORR(RANK(OPEN), RANK(MEAN(VOLUME,15)), 21), 9), 7), 8)))

In [ ]:
Alpha120 = (RANK((VWAP - CLOSE)) / RANK((VWAP + CLOSE)))

In [ ]:
Alpha121 = ((RANK((VWAP - MIN(VWAP, 12)))**TSRANK(CORR(TSRANK(VWAP, 20), TSRANK(MEAN(VOLUME,60), 2), 18), 3)) * -1)

In [ ]:
Alpha123 = ((RANK(CORR(SUM(((HIGH + LOW) / 2), 20), SUM(MEAN(VOLUME,60), 20), 9)) < RANK(CORR(LOW, VOLUME, 6))) * -1)

In [ ]:
Alpha124 = (CLOSE - VWAP) / DECAYLINEAR(RANK(TSMAX(CLOSE, 30)),2)

In [ ]:
Alpha125 = (RANK(DECAYLINEAR(CORR((VWAP), MEAN(VOLUME,80),17), 20)) / RANK(DECAYLINEAR(DELTA(((CLOSE * 0.5) + (VWAP * 0.5)), 3), 16)))

In [ ]:
Alpha126 = (CLOSE+HIGH+LOW)/3

In [ ]:
'''这里mean少给了一个参数 窗口n'''
#Alpha127 = (MEAN((100*(CLOSE-MAX(CLOSE,12))/(MAX(CLOSE,12)))**2))**(1/2)

In [ ]:
Alpha130 = (RANK(DECAYLINEAR(CORR(((HIGH + LOW) / 2),MEAN(VOLUME,40),9),10))/RANK(DECAYLINEAR(CORR(RANK(VWAP), RANK(VOLUME), 7),3)))

In [ ]:
Alpha131 = (RANK(DELAY(VWAP, 1))**TSRANK(CORR(CLOSE,MEAN(VOLUME,50), 18), 18))

In [ ]:
Alpha132 = MEAN(AMOUNT,20)

In [ ]:
Alpha133 = ((20-HIGHDAY(HIGH,20))/20)*100-((20-LOWDAY(LOW,20))/20)*100

In [ ]:
Alpha134 = (CLOSE-DELAY(CLOSE,12))/DELAY(CLOSE,12)*VOLUME

In [ ]:
Alpha136 = ((-1 * RANK(DELTA(RET, 3))) * CORR(OPEN, VOLUME, 10))

In [ ]:
Alpha138 = ((RANK(DECAYLINEAR(DELTA((((LOW * 0.7) + (VWAP *0.3))), 3), 20))-TSRANK(DECAYLINEAR(TSRANK(CORR(TSRANK(LOW, 8), TSRANK(MEAN(VOLUME,60), 17), 5), 19), 16), 7)) * -1)

In [ ]:
Alpha139 = (-1 * CORR(OPEN, VOLUME, 10))

In [ ]:
Alpha140 = MIN(RANK(DECAYLINEAR(((RANK(OPEN) + RANK(LOW)) - (RANK(HIGH) + RANK(CLOSE))),8)),TSRANK(DECAYLINEAR(CORR(TSRANK(CLOSE, 8), TSRANK(MEAN(VOLUME,60), 20), 8), 7), 3))

In [ ]:
Alpha141 = (RANK(CORR(RANK(HIGH), RANK(MEAN(VOLUME,15)), 9))* -1)

In [ ]:
Alpha142 = (((-1 * RANK(TSRANK(CLOSE, 10))) * RANK(DELTA(DELTA(CLOSE,1), 1))) *RANK(TSRANK((VOLUME/MEAN(VOLUME,20)), 5)))

In [ ]:
#Alpha144 = SUMIF(ABS(CLOSE/DELAY(CLOSE,1)-1)/AMOUNT,20,\
#                 CLOSE<DELAY(CLOSE,1))/COUNT(CLOSE, CLOSE<DELAY(CLOSE, 1),20)

In [ ]:
Alpha145 = (MEAN(VOLUME,9)-MEAN(VOLUME,26))/MEAN(VOLUME,12)*100

In [ ]:
Alpha148 = ((RANK(CORR((OPEN), SUM(MEAN(VOLUME,60), 9), 6)) < RANK((OPEN - TSMIN(OPEN, 14)))) * -1)

In [ ]:
Alpha150 = (CLOSE+HIGH+LOW)/3*VOLUME

In [ ]:
Alpha153 = (MEAN(CLOSE,3)+MEAN(CLOSE,6)+MEAN(CLOSE,12)+MEAN(CLOSE,24))/4

In [ ]:
Alpha154 = (((VWAP - MIN(VWAP, 16))) < (CORR(VWAP, MEAN(VOLUME,180), 18)))

In [ ]:
Alpha156 = (MAX(RANK(DECAYLINEAR(DELTA(VWAP, 5), 3)), RANK(DECAYLINEAR(((DELTA(((OPEN * 0.15) + (LOW *0.85)), 2) / ((OPEN * 0.15) + (LOW * 0.85))) * -1), 3))) * -1)

In [ ]:
Alpha157 = (MIN(PROD(RANK(RANK(LOG(SUM(TSMIN(RANK(RANK((-1 * RANK(DELTA((CLOSE - 1), 5))))), 2), 1)))), 1), 5) + TSRANK(DELAY((-1 * RET), 6), 5))

In [ ]:
Alpha159 = ((CLOSE-SUM(MIN(LOW,DELAY(CLOSE,1)),6))/SUM(MAX(HIGH,DELAY(CLOSE,1))-MIN(LOW,DELAY(CLOSE,1)),6) *12*24+(CLOSE-SUM(MIN(LOW,DELAY(CLOSE,1)),12))/SUM(MAX(HIGH,DELAY(CLOSE,1))-MIN(LOW,DELAY(CLOSE,1)),12)*6*24+(CLOSE-SUM(MIN(LOW,DELAY(CLOSE,1)),24))/SUM(MAX(HIGH,DELAY(CLOSE,1))-MIN(LOW,DELAY(CLOSE,1)),24)*6*24)*100/(6*12+6*24+12*24)

In [ ]:
Alpha161 = MEAN(MAX(MAX((HIGH-LOW),ABS(DELAY(CLOSE,1)-HIGH)),ABS(DELAY(CLOSE,1)-LOW)),12)

In [ ]:
Alpha163 = RANK(((((-1 * RET) * MEAN(VOLUME,20)) * VWAP) * (HIGH - CLOSE)))

In [ ]:
'''SUMAC(A,n)没有给窗口n，故此随意设为5，但是max和min好像也只传入了一个series，因此这个alpha index写得也有问题'''
#Alpha165 = MAX(SUMAC(CLOSE-MEAN(CLOSE,48),5))-MIN(SUMAC(CLOSE-MEAN(CLOSE,48),5))/STD(CLOSE,48)

In [32]:
'''SUM((CLOSE/DELAY(CLOSE,1),20) 这个地方写的有问题'''

Alpha166 = -20*((20-1)**1.5)*SUM((CLOSE/DELAY(CLOSE,1)-MEAN(CLOSE/DELAY(CLOSE,1)-1,20)),20)\
            /((20-1)*(20-2)*(SUM((CLOSE/DELAY(CLOSE,1),20)**2,20))**1.5)
#多了个窗口

TypeError: unsupported operand type(s) for ** or pow(): 'tuple' and 'int'

In [ ]:
Alpha168 = (-1*VOLUME/MEAN(VOLUME,20))

In [ ]:
Alpha170 = ((((RANK((1 / CLOSE)) * VOLUME) / MEAN(VOLUME,20)) * ((HIGH * RANK((HIGH - CLOSE))) / (SUM(HIGH, 5) / 5))) - RANK((VWAP - DELAY(VWAP, 5))))

In [ ]:
Alpha171 = ((-1 * ((LOW - CLOSE) * (OPEN**5))) / ((CLOSE - HIGH) * (CLOSE**5)))

In [ ]:
Alpha175 = MEAN(MAX(MAX((HIGH-LOW),ABS(DELAY(CLOSE,1)-HIGH)),ABS(DELAY(CLOSE,1)-LOW)),6)

In [ ]:
Alpha176 = CORR(RANK(((CLOSE - TSMIN(LOW, 12)) / (TSMAX(HIGH, 12) - TSMIN(LOW,12)))), RANK(VOLUME), 6)

In [ ]:
Alpha177 = ((20-HIGHDAY(HIGH,20))/20)*100

In [ ]:
Alpha178 = (CLOSE-DELAY(CLOSE,1))/DELAY(CLOSE,1)*VOLUME

In [ ]:
Alpha179 = (RANK(CORR(VWAP, VOLUME, 4)) *RANK(CORR(RANK(LOW), RANK(MEAN(VOLUME,50)), 12)))

In [ ]:
'''SUMAC missing 1 required positional argument: n'''
#Alpha183 = MAX(SUMAC(CLOSE-MEAN(CLOSE,24)))-MIN(SUMAC(CLOSE-MEAN(CLOSE,24)))/STD(CLOSE,24)

In [ ]:
Alpha184 = (RANK(CORR(DELAY((OPEN - CLOSE), 1), CLOSE, 200)) + RANK((OPEN - CLOSE)))

In [ ]:
Alpha185 = RANK((-1 * ((1 - (OPEN / CLOSE))**2)))

In [ ]:
Alpha189 = MEAN(ABS(CLOSE-MEAN(CLOSE,6)),6)

In [ ]:
'''invalid syntax'''
#Alpha190 = np.log((COUNT(CLOSE/DELAY(CLOSE)-1>((CLOSE/DELAY(CLOSE,19))**(1/20)-1),20)-1)*(SUMIF(((CLOSE/DELAY(CLOSE,1)-(CLOSE/DELAY(CLOSE,19))**(1/20)-1))**2,20,CLOSE/DELAY(CLOSE,1)<(CLOSE/DELAY(CLOSE,19))**(1/20)1))/((COUNT((CLOSE/DELAY(CLOSE,1)<(CLOSE/DELAY(CLOSE,19))**(1/20)-1),20))*(SUMIF((CLOSE/DELAY(CLOSE,1)-((CLOSE/DELAY(CLOSE,19))**(1/20)-1))**2,20,CLOSE/DELAY(CLOSE,1)>(CLOSE/DELAY(CLOSE,19))**(1/20)-1))))

In [ ]:
Alpha191 = ((CORR(MEAN(VOLUME,20), LOW, 5) + ((HIGH + LOW) / 2)) - CLOSE)

In [ ]:
rolling 都加上min_periods = 1, ffill

rank() 输出他的序数

SUMAC() 自己手动加一个窗口

pandas iter machine something

判断信号好不好的方法是，看未来五分钟的收益率，作为target
用pearson correlation来看每个信号跟收益率相关性有多少

设置网格搜索 寻找最优参数

设计一个 搜索最优

Alpha-index线性组合，求他前面最大的参数

搜索，如果搜到了就把新的字符串返回，判断是不是数字，判断这个alpha里面有多少个变量是可以需要修改的


eval() 函数，可以自动执行这段字符串 然后得到他的结果